In [1]:
import time

notebook_start_time = time.time()

# Set up environment

In [2]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment
Adding the following directory to the PYTHONPATH: /Users/pauliusztin/Documents/01_projects/hopsworks_recsys/hands-on-recommender-system


# Inference pipeline: Deploying and testing the inference pipeline 

In this notebook, we will dig into the inference pipeline and deploy it to Hopsworks as a real-time service.

## 📝 Imports

In [3]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [4]:
project, fs = hopsworks_integration.get_feature_store()

2024-11-21 15:28:38.691 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:12 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/15551
Connected. Call `.close()` to terminate connection gracefully.


# Deploying the ranking inference pipeline


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it:

In [5]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(
    project=project
)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 0.000%|          | 0/4306 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1117 elapsed<00:00 remaining<?

Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.
Before making predictions, start the deployment by using `.start()`


Now, we have to explicitly start the deployment:

In [18]:
ranking_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [7]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335876

Instance name: ranking-transformer-default-00001-deployment-6774d48698-9x9kd
2024-11-21 13:28:57.010 7 root INFO [<module>():180] Loading serving script
2024-11-21 13:29:00.025 7 root INFO [__init__():117] Initializing transformer for deployment: ranking
Connected. Call `.close()` to terminate connection gracefully.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/common/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2024-11-21 13:29:09.730 7 root INFO 

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [8]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

Let's define a dummy test example to test our ranking deployment (only the `customer_id` has to match):

In [9]:
test_ranking_input = {
    "instances": [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]
}

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates, k=3)
recommendations

['372860002', '688105013', '525518005']

Check logs in case of failure:

In [19]:
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335876

Instance name: ranking-transformer-default-00001-deployment-67f94444d8-4qgks
2024-11-21 14:56:39.110 7 root INFO [<module>():180] Loading serving script
2024-11-21 14:56:42.123 7 root INFO [__init__():117] Initializing transformer for deployment: ranking
Connected. Call `.close()` to terminate connection gracefully.
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/common/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-RC0/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2024-11-21 14:56:51.807 7 root INFO 

# Deploying the query inference pipeline

In [11]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy(project=project)
)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 0.000%|          | 0/2173 elapsed<00:00 remaining<?

Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.
Before making predictions, start the deployment by using `.start()`


At this point, you have registered your deployment. To start it up you need to run:

In [12]:
query_model_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [13]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer", tail=20)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335877

Instance name: query-transformer-default-00001-deployment-b967dbd5d-jzcvq
2024-11-21 13:29:37.781 7 root INFO [<module>():180] Loading serving script
2024-11-21 13:29:40.881 7 root INFO [__init__():117] Initializing transformer for deployment: query
Connected. Call `.close()` to terminate connection gracefully.
2024-11-21 13:29:41.948 7 root INFO [<module>():196] Starting KServe server
2024-11-21 13:29:41.949 7 root INFO [register_model():187] Registering model: query
2024-11-21 13:29:41.949 7 root INFO [start():129] Setting max asyncio worker threads as 12
2024-11-21 13:29:41.949 7 root INFO [serve():139] Starting uvicorn with 1 workers
2024-11-21 13:29:41.973 7 uvicorn.error INFO [serve():84] Started server process [7]
2024-11-21 13:29:41.973 7 uvicorn.error INFO [startup():45] Waiting for application startup.
2024-11-21 13:29:41.975 7 root INFO [start():62] Starting gRPC

## <span style="color:#ff5f27"> Testing the inference pipeline </span>

Define a test input example:

In [14]:
data = {
    "instances": {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
}

['474461002', '824341001', '597639006']

Test out the deployment:

In [ ]:
ranked_candidates = query_model_deployment.predict(data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

Check logs in case of failure:

In [15]:
query_model_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/15551/deployments/335877

Instance name: query-transformer-default-00001-deployment-b967dbd5d-jzcvq
2024-11-21 13:29:37.781 7 root INFO [<module>():180] Loading serving script
2024-11-21 13:29:40.881 7 root INFO [__init__():117] Initializing transformer for deployment: query
Connected. Call `.close()` to terminate connection gracefully.
2024-11-21 13:29:41.948 7 root INFO [<module>():196] Starting KServe server
2024-11-21 13:29:41.949 7 root INFO [register_model():187] Registering model: query
2024-11-21 13:29:41.949 7 root INFO [start():129] Setting max asyncio worker threads as 12
2024-11-21 13:29:41.949 7 root INFO [serve():139] Starting uvicorn with 1 workers
2024-11-21 13:29:41.973 7 uvicorn.error INFO [serve():84] Started server process [7]
2024-11-21 13:29:41.973 7 uvicorn.error INFO [startup():45] Waiting for application startup.
2024-11-21 13:29:41.975 7 root INFO [start():62] Starting gRPC

# <span style="color:#ff5f27"> Stopping the Hopsworks deployments </span>

Stop the deployment when you're not using it.

In [16]:
ranking_deployment.stop()
query_model_deployment.stop()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

## <span style="color:#ff5f27"> Inspecting the deployments in Hopsworks UI </span>

Go to [Hopsworks UI](https://www.hopsworks.ai/), **Data Science → Deployments** section and inspect the newly created deployments.

---

In [17]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2024-11-21 15:30:16.522 | INFO     | __main__:<module>:4 - ⌛️ Notebook Execution time: 102.80 seconds ~ 1.71 minutes


# <span style="color:#ff5f27">→ Next Steps </span>

The last step is to schedule the materialization jobs.